In [3]:
import pandas as pd
import numpy as np
import json

In [4]:
train_data1_pd = pd.read_json("data1/training_set_task2.txt")
train_data2_pd = pd.read_csv("data2/datasets1/new_data2.csv")

In [5]:
train_data1_pd = train_data1_pd.drop(columns=["id"])
train_data1_pd

,text,labels
0,THERE ARE ONLY TWO GENDERS\n\nFEMALE \n\nMALE\n,"[{'start': 0, 'end': 41, 'technique': 'Black-a..."
1,This is not an accident!,[]
2,SO BERNIE BROS HAVEN'T COMMITTED VIOLENCE EH?\...,"[{'end': 83, 'start': 47, 'technique': 'Slogan..."
3,PATHETIC\n\nThe Cowardly Asshole\nWeak Failure...,"[{'end': 8, 'start': 0, 'technique': 'Loaded L..."
4,WHO TRUMP REPRESENTS\n\nWHO DEMOCRATS REPRESENT\n,[]
...,...,...
683,"HOW TO KILL THE CORONOVIRUS...\nHey Hillary, I...","[{'start': 0, 'end': 99, 'technique': 'Smears'..."
684,"LOOK, WHEN I'M IN THE WAFFLE HOUSE, I'M GOING ...",[]
685,I support MLB\n\nIt's BLM you idiot,"[{'start': 28, 'end': 33, 'technique': 'Loaded..."
686,"PELOSI ENDORSES BIDEN\n""He is a leader who is ...","[{'start': 45, 'end': 121, 'technique': 'Loade..."


In [6]:
train_data2_pd 

,text,labels
0,"David Duke, the white supremacist icon and for...","[{'start': 16, 'end': 38, 'technique': 'Name c..."
1,"However, one individual who represents the Mus...","[{'start': 97, 'end': 105, 'technique': 'Loade..."
2,Three of the videos were put out by Duke’s off...,"[{'start': 127, 'end': 150, 'technique': 'Name..."
3,"At this same rally, a white nationalist plowed...","[{'start': 22, 'end': 120, 'technique': 'Appea..."
4,"In February 2016, Zakkout publicized on his Fa...","[{'start': 97, 'end': 209, 'technique': 'Appea..."
...,...,...
8992,“Policies like background checks on all gun sa...,"[{'start': 54, 'end': 78, 'technique': 'Slogan..."
8993,The group also passed a resolution backing red...,"[{'start': 43, 'end': 56, 'technique': 'Repeti..."
8994,“Policies like background checks on all gun sa...,"[{'start': 54, 'end': 67, 'technique': 'Repeti..."
8995,“The U.S. Conference of Mayors will continue d...,"[{'start': 83, 'end': 98, 'technique': 'Flag-w..."


In [7]:
def preproc_json(x):
    x = x.replace("'", '"')
    x = x.replace('Someone"s', "Someone's")
    x = json.loads(x)
    return x

train_data2_pd["labels"] = train_data2_pd["labels"].apply(lambda x: preproc_json(x))

In [8]:
train_pd = pd.concat([train_data1_pd, train_data2_pd], ignore_index=True)

In [9]:
train_pd

,text,labels
0,THERE ARE ONLY TWO GENDERS\n\nFEMALE \n\nMALE\n,"[{'start': 0, 'end': 41, 'technique': 'Black-a..."
1,This is not an accident!,[]
2,SO BERNIE BROS HAVEN'T COMMITTED VIOLENCE EH?\...,"[{'end': 83, 'start': 47, 'technique': 'Slogan..."
3,PATHETIC\n\nThe Cowardly Asshole\nWeak Failure...,"[{'end': 8, 'start': 0, 'technique': 'Loaded L..."
4,WHO TRUMP REPRESENTS\n\nWHO DEMOCRATS REPRESENT\n,[]
...,...,...
9680,“Policies like background checks on all gun sa...,"[{'start': 54, 'end': 78, 'technique': 'Slogan..."
9681,The group also passed a resolution backing red...,"[{'start': 43, 'end': 56, 'technique': 'Repeti..."
9682,“Policies like background checks on all gun sa...,"[{'start': 54, 'end': 67, 'technique': 'Repeti..."
9683,“The U.S. Conference of Mayors will continue d...,"[{'start': 83, 'end': 98, 'technique': 'Flag-w..."


In [10]:
labels = [
    "Reductio ad hitlerum",
    "Whataboutism",
    "Presenting Irrelevant Data (Red Herring)",
    "Doubt",
    "Slogans",
    "Appeal to fear/prejudice",
    "Obfuscation, Intentional vagueness, Confusion",
    "Misrepresentation of Someone's Position (Straw Man)",
    "Glittering generalities (Virtue)",
    "Appeal to authority",
    "Repetition",
    "Bandwagon",
    "Causal Oversimplification",
    "Name calling/Labeling",
    "Thought-terminating cliché",
    "Flag-waving",
    "Exaggeration/Minimisation",
    "Smears",
    "Loaded Language",
    "Black-and-white Fallacy/Dictatorship"
]

In [11]:
def separation_of_topics(data):
    texts = []
    prop_mask = []
    techniques = []
    for i in range(data.shape[0]):
        text = data["text"][i]
        dict_prop = dict()
        for label in data["labels"][i]:
            if label == '[':
                print(data["labels"][i], len(data["labels"][i]))
            technique = label["technique"]
            if technique not in dict_prop:
                dict_prop[technique] = np.array([0] * len(text))
            dict_prop[technique][label["start"]:label["end"]] = 1
        #for technique in labels:
        #    if technique not in dict_prop:
        #        dict_prop[technique] = np.array([0] * len(text))
        for key in dict_prop:
            texts.append(text)
            techniques.append(key)
            prop_mask.append(dict_prop[key])
    return texts, prop_mask, techniques

In [15]:
texts, prop_mask, techniques = separation_of_topics(train_pd)
sep_train_pd = pd.DataFrame(np.array([texts, prop_mask, techniques], dtype=object).T,
                            columns = ["text", "prop_mask", "technique"])

In [19]:
sep_train_pd.to_csv("data.csv", index=False)